In [4]:
#! pip install requests
#! pip install pydata_google_auth
#! pip install google.cloud
import pandas as pd 
import requests
import os 
import pydata_google_auth
from google.oauth2 import service_account
from google.cloud import language
import json 
from google.cloud import storage 
from google.cloud import bigquery

# Main function. 

def get_today_news():
    
    # Authenticate to GCP. 
  
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=''

    # Make initial request to Reddit API to return a token.

    auth = requests.auth.HTTPBasicAuth('CLIENT_ID', 'SECRET_TOKEN')
    data = {
    'grant_type': 'client_credentials',
    'username': 'username',
    'password': 'password'
    }
    headers = {'User-Agent': 'News/0.0.1'}
    request = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
    token = request.json()['access_token']
    headers = {**headers, **{'Authorization': f"bearer {token}"}}
    
    # Make the request to the desired subreddit: r/news. 

    news_requests = requests.get('https://oauth.reddit.com/r/news/hot', headers=headers, params={'limit': '100'})

    # Transformations. 

    # Loop through the returned JSON body to create our columns.

    df = pd.DataFrame()
    for post in news_requests.json()['data']['children']:
        df = df.append({
        'title': post['data']['title'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'score': post['data']['score'],
        'ups': post['data']['ups'],
        'domain': post['data']['domain'],
        'num_comments': post['data']['num_comments']
    }, ignore_index=True)
    
    # Establish a BigQuery client. 

    client = bigquery.Client()
    news_dataset_id = 'reddit_news'
    news_table_id = 'r_news'
    
    news_ref = client.dataset(news_dataset_id)
    news_table_id = news_ref.table(news_table_id)
    
    # Configure the load job.

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition='WRITE_TRUNCATE'
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect=True
    job_config.ignore_unknown_values=True 

    # Load the data to the "r_news" table. 

    job = client.load_table_from_dataframe(
    df,
    news_table_id,
    location='US',
    job_config=job_config)

    job.result()

    # Print a message when complete. 

    print('News table loaded.')

ModuleNotFoundError: No module named 'google.cloud'